<a href="https://colab.research.google.com/github/rickiepark/fine-tuning-llm/blob/main/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3장 LoRA

사용된 패키지

* torch 2.9.0
* transformers 5.2.0
* datasets 4.0.0
* bitsandbytes 0.49.1
* peft 1.18.0

In [1]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
import os
from transformers.utils import logging

# tqdm 비활성화
os.environ["DISABLE_TQDM"] = "1"
logging.disable_progress_bar()

### 스포일러
이 장에서는 다음과 같은 내용을 배웁니다.

- LoRA(Low-Rank Adaptation)가 무엇이고 왜 유용한지 이해합니다.
- 훈련을 위해 양자화된 모델을 준비합니다.
- `peft` 라이브러리를 사용해 어댑터(adapter)를 만들어 베이스 모델에 추가합니다.
- 훈련 대상 층을 지정하기 위한 설정 옵션을 설명합니다.

### 패키지 설치

In [2]:
# 코랩에서 실행하는 경우
!pip install bitsandbytes trl

### 라이브러리 임포트

In [3]:
import numpy as np
import torch
import torch.nn as nn
from copy import deepcopy
from numpy.linalg import matrix_rank
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

### 목표

LLM에 있는 거대 선형 층에 어댑터를 추가하여 훈련 파라미터 개수를 극적으로 줄입니다. 훈련 파라미터 개수를 원래 파라미터 개수의 1% 미만으로 쉽게 줄일 수 있습니다. 계산(그레이디언트가 적음)과 메모리 사용량(옵티마이저가 추적할 파라미터가 적음)을 줄임으로써 효율성을 크게 높일 수 있습니다. 하지만 LoRA는 완전히 미세 튜닝한 모델의 성능에 도달하지 못할 가능성이 많으며 효과는 베이스 모델과 작업에 따라 다를 수 있음을 유념하세요.

### 준비 운동

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/matmul.png?raw=True)
<center>그림 3.1 - 행렬 곱셈</center>

### LoRA의 핵심

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/two_matrices.png?raw=True)
<center>그림 3.2 - 두 개의 작은 행렬 곱하기</center>

In [4]:
base_layer = nn.Linear(1024, 1024, bias=False)
base_layer.weight.shape, base_layer.weight.numel()

(torch.Size([1024, 1024]), 1048576)

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/lowrank_matrices.png?raw=True)
<center>그림 3.3 동결된 가중치와 작은 행렬</center>

In [5]:
torch.manual_seed(11)
r = 8
layer_A = nn.Linear(base_layer.in_features, r, bias=False)
layer_B = nn.Linear(r, base_layer.out_features, bias=False)
layer_A, layer_B

(Linear(in_features=1024, out_features=8, bias=False),
 Linear(in_features=8, out_features=1024, bias=False))

In [6]:
layer_A.weight.numel(), layer_B.weight.numel()

(8192, 8192)

In [7]:
composite = layer_B.weight @ layer_A.weight
composite.shape, composite.numel()

(torch.Size([1024, 1024]), 1048576)

In [8]:
matrix_rank(composite.detach().numpy())

np.int64(8)

$$
\Large
\text{output} = X @ (W + B @ A)^T
$$
<center>식 3.1 가중치에 곱셈 결과 행렬을 더하기</center>

In [9]:
torch.manual_seed(19)
batch = torch.randn(1, 1024)

batch @ (base_layer.weight.data + layer_B.weight @ layer_A.weight).T

tensor([[-0.9942, -0.8913, -0.6378,  ...,  0.3953, -0.8154,  1.1929]],
       grad_fn=<MmBackward0>)

$$
\Large
\text{output} = \underbrace{X @ W^T}_{O_W} + \underbrace{X @ (B @ A)^T}_{O_{AB}}
$$
<center>식 3.2 두 개의 정방향 계산</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/forward.png?raw=True)
<center>그림 3.4 두 개의 정방향 계산 사용하기</center>

In [10]:
regular_output = batch @ base_layer.weight.data.T
additional_output = batch @ (layer_B.weight @ layer_A.weight).T
regular_output, additional_output

(tensor([[-0.9584, -0.9143, -0.1771,  ...,  0.6873, -1.0098,  0.6888]]),
 tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
        grad_fn=<MmBackward0>))

$$
\Large
\text{additional} = X @ (B @ A)^T = \underbrace{\underbrace{(X @ A^T)}_{O_A} @ B^T}_{O_{AB}}
$$
<center>식 3.3 어댑터의 정방향 계산을 분할하기</center>

In [11]:
out_A = (batch @ layer_A.weight.T)
additional_output = out_A @ layer_B.weight.T
additional_output

tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
       grad_fn=<MmBackward0>)

In [12]:
regular_output = base_layer(batch)
out_A = layer_A(batch)
additional_output = layer_B(out_A)
output = regular_output + additional_output
regular_output, additional_output, output

(tensor([[-0.9584, -0.9143, -0.1771,  ...,  0.6873, -1.0098,  0.6888]],
        grad_fn=<MmBackward0>),
 tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
        grad_fn=<MmBackward0>),
 tensor([[-0.9942, -0.8913, -0.6378,  ...,  0.3953, -0.8154,  1.1929]],
        grad_fn=<AddBackward0>))

$$
\Large
\text{output} = X @ W^T + \frac{\alpha}{r}\left[X @ (B @ A)^T\right]
$$
<center>식 3.4 LoRA의 스케일링 인자</center>

In [13]:
alpha = 2*r
output = regular_output + (alpha / r) * additional_output
output

tensor([[-1.0299, -0.8683, -1.0985,  ...,  0.1032, -0.6210,  1.6970]],
       grad_fn=<AddBackward0>)

### 준비 코드

In [14]:
supported = torch.cuda.is_bf16_supported(including_emulation=False)
compute_dtype = (torch.bfloat16 if supported else torch.float32)

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=compute_dtype
)

model_q4 = AutoModelForCausalLM.from_pretrained("facebook/opt-350m",
                                                device_map='cuda:0',
                                                dtype=compute_dtype,
                                                quantization_config=nf4_config)

### 파라미터 타입과 그레이디언트

****
**"파라미터 타입과 그레이디언트" 요약**

- 양자화는 양자화된 선형 층만 동결합니다.
- 양자화 후에 `prepare_model_for_kbit_training()` 함수를 사용해 모델을 준비합니다.
    - 모든 층을 동결합니다.
    - 양자화되지 않은 모든 16비트 층을 FP32로 바꾸어 훈련을 안정화합니다.
    - 그레이디언트 체크포인팅을 활성화합니다.
- LoRA 설정을 사용해 나중에 선택적으로 층을 동결 해제할 수 있습니다.
****

In [15]:
def trainable_parms(model):
    parms = [(name, param.dtype) for name, param in model.named_parameters() if param.requires_grad]
    return parms

trainable_parms(model_q4.model)

[('decoder.embed_tokens.weight', torch.float32),
 ('decoder.embed_positions.weight', torch.float32),
 ('decoder.layers.0.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.0.self_attn_layer_norm.bias', torch.float32),
 ('decoder.layers.0.final_layer_norm.weight', torch.float32),
 ('decoder.layers.0.final_layer_norm.bias', torch.float32),
 ('decoder.layers.1.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.1.self_attn_layer_norm.bias', torch.float32),
 ('decoder.layers.1.final_layer_norm.weight', torch.float32),
 ('decoder.layers.1.final_layer_norm.bias', torch.float32),
 ('decoder.layers.2.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.2.self_attn_layer_norm.bias', torch.float32),
 ('decoder.layers.2.final_layer_norm.weight', torch.float32),
 ('decoder.layers.2.final_layer_norm.bias', torch.float32),
 ('decoder.layers.3.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.3.self_attn_layer_norm.bias', torch.float32),
 ('decoder.la

#### `prepare_model_for_kbit_training()`

In [16]:
prepared_model = prepare_model_for_kbit_training(model_q4,
                                        use_gradient_checkpointing=True,
                                        gradient_checkpointing_kwargs={'use_reentrant': False})
prepared_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
      (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (

In [17]:
trainable_parms(prepared_model)

[]

In [18]:
def parms_of_dtype(model, dtype=torch.float32):
    parms = [name for name, param in model.named_parameters() if param.dtype == dtype]
    return parms

In [19]:
parms_of_dtype(prepared_model)

['model.decoder.embed_tokens.weight',
 'model.decoder.embed_positions.weight',
 'model.decoder.layers.0.self_attn.k_proj.bias',
 'model.decoder.layers.0.self_attn.v_proj.bias',
 'model.decoder.layers.0.self_attn.q_proj.bias',
 'model.decoder.layers.0.self_attn.out_proj.bias',
 'model.decoder.layers.0.self_attn_layer_norm.weight',
 'model.decoder.layers.0.self_attn_layer_norm.bias',
 'model.decoder.layers.0.fc1.bias',
 'model.decoder.layers.0.fc2.bias',
 'model.decoder.layers.0.final_layer_norm.weight',
 'model.decoder.layers.0.final_layer_norm.bias',
 'model.decoder.layers.1.self_attn.k_proj.bias',
 'model.decoder.layers.1.self_attn.v_proj.bias',
 'model.decoder.layers.1.self_attn.q_proj.bias',
 'model.decoder.layers.1.self_attn.out_proj.bias',
 'model.decoder.layers.1.self_attn_layer_norm.weight',
 'model.decoder.layers.1.self_attn_layer_norm.bias',
 'model.decoder.layers.1.fc1.bias',
 'model.decoder.layers.1.fc2.bias',
 'model.decoder.layers.1.final_layer_norm.weight',
 'model.decode

In [20]:
prepared_model.get_memory_footprint()/1e6

264.15104

### PEFT

"_🤗 PEFT(Parameter-Efficient Fine-Tuning)는 사전 훈련된 대규모 모델을 다양한 후속 애플리케이션에 효율적으로 적응시키기 위한 라이브러리입니다. 매우 많은 비용이 들기 때문에 모델 파라미터를 모두 미세 튜닝하지 않습니다. PEFT 방법은 모델에 추가된 적은 개수의 파라미터만 미세 튜닝합니다. 계산 비용과 저장 공간을 크게 줄이면서 완전히 미세 튜닝된 모델에 비견할만한 성능을 냅니다. 이를 통해 대규모 언어 모델을 개인용 하드웨어에서 훈련할 수 있습니다._

_PEFT는 transformers, diffusers, accelerate 라이브러리와 통합되어 있어 대규모 모델을 쉽고 빠르게 로드하고, 훈련하고, 추론에 사용할 수 있습니다._"

****
**"PEFT" 요약**
- 다음 기본 설정이 대부분의 경우 잘 동작합니다.
```python
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(model, config)
```
    - 일반적인 랭크 값은 8, 16, 32이지만 더 높은 값을 사용해도 모델의 메모리 사용량에 큰 영향을 미치지 않습니다.
    - 스케일링 인자인 `lora_alpha`는 일반적으로 랭크의 두 배로 합니다.
- 모델이 `Conv1D`를 사용한다면 설정에 `fan_in_fan_out=True`를 추가합니다.
- 최근에 공개된 모델이라면 수동으로 `target_modules`를 지정해야 할 수 있습니다.
  - 일반적으로 어텐션 모듈 안에 있는 대규모의 선형 층을 지정합니다.
- 기본적으로 어댑터만 훈련 가능합니다.
  - 층 정규화 같은 다른 층을 훈련하고 싶다면 `modules_to_save` 매개변수에 추가하세요.
  - 토크나이저에 토큰을 추가하고 싶다면 임베딩과 모델 헤드와 같이 어휘사전에 관련된 층도 훈련해야 합니다.
****

In [21]:
lora_config = LoraConfig()
lora_config

LoraConfig(task_type=None, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, peft_version='0.18.0', base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules=None, exclude_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, alora_invocation_tokens=None, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None, arrow_config=None, ensure_weight_tying=False)

In [22]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

### `target_modules`

매주 새로운 모델과 구조가 공개되고 있기 때문에 현재 설치된 PEFT 라이브러리의 사전 설정된 층 목록에 없을 수 있습니다. 이런 경우 다음과 같은 오류가 발생합니다:

***
`ValueError: Please specify `target_modules` in `peft_config``
***

이름이 확인되면 `target_modules` 매개변수를 사용하여 어댑터를 적용할 모듈 이름이나 목록을 지정할 수 있습니다.

**지원 모델**

사용하려는 모델 구조가 peft 패키지에서 지원하는지 확인하려면 다음 코드를 참고하세요:

In [23]:
from peft.utils.constants import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING.keys()

dict_keys(['t5', 'mt5', 'bart', 'gpt2', 'bloom', 'blip-2', 'opt', 'gptj', 'gpt_neox', 'gpt_neo', 'bert', 'roberta', 'xlm-roberta', 'electra', 'deberta-v2', 'deberta', 'layoutlm', 'llama', 'llama4', 'chatglm', 'gpt_bigcode', 'mpt', 'RefinedWebModel', 'RefinedWeb', 'falcon', 'btlm', 'codegen', 'mistral', 'mixtral', 'stablelm', 'phi', 'gemma', 'gemma2', 'gemma3_text', 'qwen2', 'qwen3', 'rwkv', 'rwkv7'])

In [24]:
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['phi']

['q_proj', 'v_proj', 'fc1', 'fc2']

#### PEFT 모델

In [25]:
peft_model = get_peft_model(prepared_model, config, adapter_name='default')
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 512, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
          (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
          (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
          (layers): ModuleList(
            (0-23): 24 x OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (v_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_fea

In [26]:
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['opt']

['q_proj', 'v_proj']

In [27]:
lin = peft_model.base_model.model.model.decoder.layers[0].self_attn.q_proj
lin

lora.Linear4bit(
  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=1024, out_features=8, bias=False)
  )
  (lora_B): ModuleDict(
    (default): Linear(in_features=8, out_features=1024, bias=False)
  )
  (lora_embedding_A): ParameterDict()
  (lora_embedding_B): ParameterDict()
  (lora_magnitude_vector): ModuleDict()
)

In [28]:
peft_model.print_trainable_parameters()

trainable params: 786,432 || all params: 331,982,848 || trainable%: 0.2369


In [29]:
trainable_parms(peft_model.base_model.model)

[('model.decoder.layers.0.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.q_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.q_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.q_proj.lora_B.default.weight'

#### `modules_to_save`

In [30]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=['layer_norm']
)

In [31]:
# `get_peft_model()` 함수는 입력된 모델을 수정하므로 다른 설정 객체로 반복해서 호출하면 설정이 꼬이게 됩니다.
# 일반적인 워크플로에서는 설정을 한 번만 로드하므로 이런 코드가 필요하지 않습니다.
_ = peft_model.unload()

In [32]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 884,736 || all params: 332,081,152 || trainable%: 0.2664


#### 임베딩

In [33]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=['layer_norm', 'embed_tokens']
)

In [34]:
# `get_peft_model()` 함수는 입력된 모델을 수정하므로 다른 설정 객체로 반복해서 호출하면 설정이 꼬이게 됩니다.
# 일반적인 워크플로에서는 설정을 한 번만 로드하므로 이런 코드가 필요하지 않습니다.
_ = peft_model.unload()

In [35]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:1222: UserWarning: Model has `tie_word_embeddings=True` and a tied layer is part of the adapter, but `ensure_weight_tying` is not set to True. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. Check the discussion here: https://github.com/huggingface/peft/issues/2777
  warnings.warn(msg)


trainable params: 26,624,000 || all params: 357,820,416 || trainable%: 7.4406


In [36]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['embed_tokens', 'q_proj', 'v_proj']
)

In [37]:
# `get_peft_model()` 함수는 입력된 모델을 수정하므로 다른 설정 객체로 반복해서 호출하면 설정이 꼬이게 됩니다.
# 일반적인 워크플로에서는 설정을 한 번만 로드하므로 이런 코드가 필요하지 않습니다.
_ = peft_model.unload()

In [38]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()

trainable params: 1,192,704 || all params: 358,128,384 || trainable%: 0.3330


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:916: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.decoder.embed_tokens'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [39]:
lin = peft_model.base_model.model.model.decoder.embed_tokens
lin

lora.Embedding(
  (base_layer): Embedding(50272, 512, padding_idx=1)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict()
  (lora_B): ModuleDict()
  (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 8x50272 (cuda:0)])
  (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 512x8 (cuda:0)])
  (lora_magnitude_vector): ModuleDict()
)

#### 어댑터 관리

In [40]:
peft_model.load_adapter('dvgodoy/opt-350m-lora-yoda', adapter_name='yoda')
lora_A = peft_model.base_model.model.model.decoder.layers[0].self_attn.q_proj.lora_A
lora_A

ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [41]:
peft_model.add_adapter(adapter_name='third', peft_config=config)
lora_A

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:916: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.decoder.embed_tokens', 'model.decoder.embed_tokens'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
  (third): Linear(in_features=1024, out_features=8, bias=False)
)

In [42]:
peft_model.delete_adapter(adapter_name='third')
lora_A

ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [43]:
peft_model.peft_config.keys()

dict_keys(['default', 'yoda'])

In [44]:
peft_model.active_adapter

'default'

In [45]:
peft_model.set_adapter('yoda')
peft_model.active_adapter

'yoda'

```python
with peft_model.disable_adapter():
    original_outputs = peft_model(inputs)

original_outputs = peft_model.base_model(inputs)
```

In [46]:
peft_model.merge_adapter(adapter_names=['yoda'])
lora_A

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:569: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.decoder.embed_tokens', 'model.decoder.embed_tokens'] are part of the adapter. This can lead to complications. You can opt to merge the adapter after cloning the weights (to untie the embeddings). You can untie the embeddings by loading the model with `tie_word_embeddings=False`. For example:
```python
from transformers import AutoModelForCausalLM

# Load original tied model
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", tie_word_embeddings=False)

# Set the randomly initialized lm_head to the previously tied embeddings
model.lm_head.weight.data = model.model.embed_tokens.weight.data.clone()

# Save the untied model
untied_model_dir = "dir/for/untied/model"
model.save_pretrained(untied_model_dir)
model.config.save_pretrained(untied_model_dir)

# Now use the original model but in untied format
model = AutoMod

ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [47]:
peft_model.unload()
peft_model.base_model.model.model.decoder.layers[0].self_attn

OPTAttention(
  (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
)

### 다음 장에서는

LoRA 어댑터는 빠르고 적은 비용으로 LLM을 미세 튜닝할 수 있습니다. 이런 거대 모델은 강력하지만 다음 토큰 예측이라는 단일 작업에 특화되어 있어 입력 구조에 한계가 있습니다. 이런 모델에게 대화 기능을 부여하려면 새로운 종류의 입력을 개발해야 합니다. 다음 장에서 채팅 템플릿에 관해 자세히 알아 보겠습니다.